In [8]:
import random
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from datetime import datetime
import pandas as pd
import time

MAIN_URL = 'https://www.elempleo.com/co/ofertas-empleo/ayer'
MAIN_boxes = '//div[@class="result-item"]'
XPATH_link = './/a[@class="text-ellipsis js-offer-title"]'
XPATH_name = './/a[@class="text-ellipsis js-offer-title"]'
XPATH_city = './/span[@class="info-city js-offer-city"]'
XPATH_summary = './/span[@class="text-primary info-salary js-offer-salary"]'
XPATH_company = './/span[@class="info-company-name js-offer-company"]'
XPATH_date = './/span[@class="info-publish-date pull-right js-offer-date"]'
XPATH_next_pag = '//nav[@class="pull-right"]/ul[@class="pagination ee-mod "]//a[@class="js-btn-next"]'
XPATH_100 = '//select[@id="ResultsByPage"]/option[@value="100"]'
XPATH_category = './/span[@class="info-tag-not-select"]'
XPATH_Num_jobs = '//nav[@class="pull-right"]/span[@class="visible-lg-inline-block results-data"]'
XPATH_cookies = '//div[@class="col-xs-12 buttons-politics text-right"]/a[@class="btn btn-default submit-politics btnAcceptPolicyNavigationCO"]'


def num_pags(jobs):
    num_p = jobs // 100
    if jobs % 100 > 0:
        num_p += 1
    return num_p

def join_categories(categories):
    categories_full = str()
    for category in categories:
        if categories_full:
            categories_full += ', ' + category.text
        else:
            categories_full += category.text
    return categories_full


def run(data):
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-extensions')
    options.add_argument('--no-sandbox')


    time.sleep(1)
    driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)
    driver.get(MAIN_URL)

    button_100 = WebDriverWait(driver, 10).until(EC.presence_of_element_located \
                                             ((By.XPATH, XPATH_100)))
    button_100.click()

    time.sleep(2)
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, XPATH_Num_jobs)))
    driver.find_element_by_xpath(XPATH_cookies).click()
    jobs = int(driver.find_element_by_xpath(XPATH_Num_jobs).text.split()[-2])
    pags = num_pags(jobs)

    #for i in range(pags):
    for i in range(3):
        
        empleos = driver.find_elements_by_xpath(MAIN_boxes)
        try:
            for empleo in empleos:
                name =  empleo.find_element_by_xpath(XPATH_name).text
                company = empleo.find_element_by_xpath(XPATH_company).text
                salary = empleo.find_element_by_xpath(XPATH_summary).text
                city = empleo.find_element_by_xpath(XPATH_city).text
                link = empleo.find_element_by_xpath(XPATH_link).get_attribute('href')
                date = empleo.find_element_by_xpath(XPATH_date).text
                date = f'{date.split()[1]}/{datetime.today().month}/{datetime.today().year}'
                category = empleo.find_elements_by_xpath(XPATH_category)
                category = join_categories(category)
                source = 'El Empleo'
                data = data.append({'Vacante': name, 'Fecha': date, 'Compania': company, 'Link': link, 'Salario': salary, 
                                    'Informacion_Adicional': category, 'Fuente': source, 'Ciudad': city}, ignore_index=True)       
        except ValueError as ve:
            print('Error el extraer la información de un empleo', ve)
    
        button_next = WebDriverWait(driver, 3).until(EC.presence_of_element_located \
                                                 ((By.XPATH, XPATH_next_pag)))
        button_next.click()
        print(f'pag {i + 1} de {pags}')
        time.sleep(random.uniform(2, 5))

    date_name = f'{datetime.today().day}-{datetime.today().month}-{datetime.today().year}'
    data.to_excel(f'Elempleo_{date_name}.xlsx', index=False)
    return data

if __name__ == '__main__':
    
    data = pd.DataFrame(columns=['Vacante', 'Fecha', 'Compania', 'Link', 'Informacion_Adicional', 'Salario',
                                       'Fuente', 'Ciudad'])
    data = run(data)
    print(data)

<ipython-input-8-a74cef97c251>:25: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)


pag 1 de 24
pag 2 de 24
pag 3 de 24
                                               Vacante      Fecha  \
0                         Profesional jr confiabilidad  17/2/2021   
1                        Auxiliar de enfermeria soacha  17/2/2021   
2    Auxiliar analista logística - auxiliar de cont...  17/2/2021   
3                  Control, programación y seguimiento  17/2/2021   
4             Asesor comercial buses y camiones bogotá  17/2/2021   
..                                                 ...        ...   
295                              Auxiliar de repuestos  17/2/2021   
296                Gerente corporativo puntos de venta  17/2/2021   
297                         Prevendedor tat - con moto  17/2/2021   
298                                Vendedor industrial  17/2/2021   
299                                  Presales engineer  17/2/2021   

                 Compania                                               Link  \
0    Empresa confidencial  https://www.elempleo.com/co/